In [1]:
"""
まずは
"""
import pandas as pd

dir = "../data/"

df = pd.read_csv(dir + "windy_value/date/value_bottom_val_date.csv", header=None)
print(df)

#dataディレクトリの下のディレクトリ
windy_value = ["date","max_wind_speed","swell","swell_spacing","time_hour","wave_height","wind_speed"]

#data/*/ディレクトリの中の*_top_*.csvファイル
value_top_data = ["value_top_val_date.csv","value_top_val_max_wind_speed.csv","value_top_val_swell.csv",
                  "value_top_val_swell_spacing.csv","value_top_val_time_hour.csv","value_top_val_wave_height.csv",
                 "value_top_val_wind_speed.csv"]

             0           1           2           3           4           5  \
0   2020-08-13  2020-08-14  2020-08-15  2020-08-16  2020-08-17  2020-08-18   
1   2020-09-01  2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06   
2   2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07   
3   2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08   
4   2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08  2020-09-09   
..         ...         ...         ...         ...         ...         ...   
69  2021-01-01  2021-01-02  2021-01-03  2021-01-04  2021-01-05  2021-01-06   
70  2021-01-01  2021-01-02  2021-01-03  2021-01-04  2021-01-05  2021-01-06   
71  2021-01-01  2021-01-02  2021-01-03  2021-01-04  2021-01-05  2021-01-06   
72  2021-01-08  2021-01-09  2021-01-10  2021-01-11  2021-01-12  2021-01-13   
73  2021-01-10  2021-01-11  2021-01-12  2021-01-13  2021-01-14  2021-01-15   

             6           7           8           9  
0   2020-0

In [2]:
eng_name_dic = {
    "taketomi_route" : ["isigaki_dep", "taketomi_dep"],
    "kurosima_route" : ["isigaki_dep", "kurosima_dep"],
    "kohama_route" : ["isigaki_dep", "tkohama_dep"],
    "iriomote_uehara_route" : ["isigaki_dep", "tuehara_dep"],
    "hatoma_route" : ["isigaki_dep", "hatoma_dep"],
    "iriomote_ohara_route" : ["isigaki_dep", "ohara_dep"],
   "hateruma_route" : ["isigaki_dep", "hateruma_dep"]
                                           } 

In [3]:
import pathlib
import glob
import re
import pprint
import os

#安栄観光の欠航データCSV
csv_file = [os.path.basename(p) for p in glob.glob(dir + '/route/taketomi_route/isigaki_dep/*', recursive=True)
     if os.path.isfile(p) and re.search(r'2020-[0|1][8|9|10|11|12]-.*',p)]
csv_file = sorted(csv_file)

print(csv_file)

#教師データのcsvファイル名の格納
#print(re.findall('(.*).csv', csv_file[0]))
date_data = re.findall('(.*).csv', csv_file[0])
# ['2020-08-01']

date_in = df[df[0].isin([date_data])]
#print(date_in)

df_bool = df[0] == date_data[0]
#print(df_bool.values.sum())
if df_bool.values.sum() == 0:
    #for文continu
    pass
else:
    #日付がある場合なら処理(windy_valueがあるということ)
    pass
#date_in

['2020-08-02.csv', '2020-08-03.csv', '2020-08-04.csv', '2020-08-05.csv', '2020-08-06.csv', '2020-08-07.csv', '2020-08-08.csv', '2020-08-09.csv', '2020-08-10.csv', '2020-08-11.csv', '2020-08-12.csv', '2020-08-13.csv', '2020-08-14.csv', '2020-08-15.csv', '2020-08-16.csv', '2020-08-17.csv', '2020-08-18.csv', '2020-08-19.csv', '2020-08-20.csv', '2020-08-21.csv', '2020-08-22.csv', '2020-08-23.csv', '2020-08-24.csv', '2020-08-25.csv', '2020-08-26.csv', '2020-08-27.csv', '2020-08-28.csv', '2020-08-29.csv', '2020-08-30.csv', '2020-08-31.csv', '2020-09-01.csv', '2020-09-02.csv', '2020-09-03.csv', '2020-09-04.csv', '2020-09-05.csv', '2020-09-06.csv', '2020-09-07.csv', '2020-09-08.csv', '2020-09-09.csv', '2020-09-10.csv', '2020-09-11.csv', '2020-09-12.csv', '2020-09-13.csv', '2020-09-14.csv', '2020-09-15.csv', '2020-09-16.csv', '2020-09-17.csv', '2020-09-18.csv', '2020-09-19.csv', '2020-09-20.csv', '2020-09-21.csv', '2020-09-22.csv', '2020-09-23.csv', '2020-09-24.csv', '2020-09-25.csv', '2020-09-

In [4]:
#windyの数値データのデータフレームをリスト格納
#now_windyはまだ
def windy_value_in(windy_value):
    """
    1番
    parameters
    ----------
    windy_value : list[str]
        windyデータの特徴量の名前
    """
    value_file_list = []
    for i in windy_value:
        value_file = glob.glob(os.path.join(dir + 'windy_value/'+i+'/', '*'))
        #print(dir + 'windy_value/'+i+'/')
        value_file = sorted(value_file)
        value_file_list.append(value_file)    #2次元配列
    #value_file_list = sorted(value_file_list)

    print(value_file_list)

    df_7 = []    #windy_valueのcsvをデータフレームにしたリスト

    for i in value_file_list:
        #print(i)
        for l, value in enumerate(i):
            print(value)
            df_7.append(pd.read_csv(value, header=None))
            #print(df)
                #df_7[0] = df
    
    return df_7

df_7 = windy_value_in(windy_value)
#print(df_7)

[['../data/windy_value/date/value_bottom_val_date.csv', '../data/windy_value/date/value_top_val_date.csv'], ['../data/windy_value/max_wind_speed/value_bottom_val_max_wind_speed.csv', '../data/windy_value/max_wind_speed/value_top_val_max_wind_speed.csv'], ['../data/windy_value/swell/value_bottom_val_swell.csv', '../data/windy_value/swell/value_top_val_swell.csv'], ['../data/windy_value/swell_spacing/value_bottom_val_swell_spacing.csv', '../data/windy_value/swell_spacing/value_top_val_swell_spacing.csv'], ['../data/windy_value/time_hour/value_bottom_val_time_hour.csv', '../data/windy_value/time_hour/value_top_val_time_hour.csv'], ['../data/windy_value/wave_height/value_bottom_val_wave_height.csv', '../data/windy_value/wave_height/value_top_val_wave_height.csv'], ['../data/windy_value/wind_speed/value_bottom_val_wind_speed.csv', '../data/windy_value/wind_speed/value_top_val_wind_speed.csv']]
../data/windy_value/date/value_bottom_val_date.csv
../data/windy_value/date/value_top_val_date.csv

In [5]:
#windyの数値データのデータフレームをリスト格納
#now_windyはまだ
def now_windy_value_in(windy_value):
    """
    1番
    parameters
    ----------
    windy_value : list[str]
        windyデータの特徴量の名前
    """
    
    df_now = []    #全てをまとめた配列
    
    for i in ["9:00","11:00","13:00","15:00","17:00"]:
        value_file_list = []
        for l in windy_value:
            value_file = glob.glob(os.path.join(dir + 'now_value_scrape/'+l+'/'+i+'/', '*'))
            #print(dir + 'now_value_scrape/'+i+'/9:00/')
            #print(value_file)
            value_file = sorted(value_file)
            value_file_list.append(value_file)    #2次元配列
        #value_file_list = sorted(value_file_list)
        #for i in value_file_list:
            #print(i)
        #print(value_file_list)

        df_now_time = []    #windy_valueのcsvをデータフレームにしたリスト

        for i in value_file_list:
            #print(i)
            for l, value in enumerate(i):
                #print(value)
                df_now_time.append(pd.read_csv(value, header=None))
                #print(df)
                    #df_7[0] = df
        df_now.append(df_now_time)
    
    return df_now

df_now = now_windy_value_in(windy_value)
#print(df_7)
print(len(df_now))


5


In [6]:
def date_value_mer_7(df_7):
    """
    2番
    日付データと数値データを結合させている
    df_7は日付の入ったdfの配列
    7:00に取得したwindy_value
    """
    df_top_value = []
    df_bottom_value = []
    for i, value in enumerate(df_7):
        #print(value)
        #print(i)
        if i == 0:    #bottom(偶数がボトム)
            df_bottom = value
        elif i == 1:    #top(奇数がトップ)
            df_top = value
        elif i%2 == 0:
            #print(df_bottom)
            df_bottom_value.append(pd.concat([df_bottom, value], axis=1, join='inner'))
            #print(df_bottom_value)
        elif i%2 == 1:
            df_top_value.append(pd.concat([df_top, value], axis=1, join='inner'))
    return df_bottom_value,df_top_value

df_bottom_value,df_top_value = date_value_mer_7(df_7)

In [7]:
def date_value_mer_now(df_now):
    """
    2番
    日付データと数値データを結合させている
    df_nowは日付の入ったdfの配列 df_now[9:00[value],7:00[value],,,17:00[value]]
    
    
    """

    df_top_value_now = []
    df_bottom_value_now = []
    for l in df_now:    #df_nowの最初の次元は時間ごとで区切られた要素
        df_top_value = []
        df_bottom_value = []
        for i, value in enumerate(l):    #特徴ごとに区切られた要素
            #print(value)
            #print(i)
            if i == 0:    #bottom(偶数がボトム)
                df_bottom = value
            elif i == 1:    #top(奇数がトップ)
                df_top = value
            elif i == 3:    #value_bottom_val_max_wind_speed_dir.csv
                df_bottom_value.append(pd.concat([df_bottom, value], axis=1, join='inner'))
            elif i == 4:    #value_top_val_max_wind_speed_dir.csv
                df_top_value.append(pd.concat([df_top, value], axis=1, join='inner'))  
            elif i == 7:    #value_bottom_val_swell_dir.csv
                df_bottom_value.append(pd.concat([df_bottom, value], axis=1, join='inner'))
            elif i == 8:    #value_top_val_swell.csv
                df_top_value.append(pd.concat([df_top, value], axis=1, join='inner'))
            elif i == 15:    #value_bottom_val_wave_height_dir.csv
                df_bottom_value.append(pd.concat([df_bottom, value], axis=1, join='inner'))
            elif i == 16:    #value_top_val_wave_height.csv
                df_top_value.append(pd.concat([df_top, value], axis=1, join='inner'))
            elif i%2 == 0:
                df_bottom_value.append(pd.concat([df_bottom, value], axis=1, join='inner'))
            elif i%2 == 1:
                df_top_value.append(pd.concat([df_top, value], axis=1, join='inner'))
        
        df_top_value_now.append(df_top_value)
        df_bottom_value_now.append(df_bottom_value)
        
    return df_bottom_value_now,df_top_value_now

df_bottom_value_now,df_top_value_now = date_value_mer_now(df_now)
print(len(df_bottom_value_now),len(df_top_value_now))
for i in df_bottom_value_now:
    print(i)
    break

5 5
[            0           1           2           3           4           5   \
0   2020-09-11  2020-09-12  2020-09-13  2020-09-14  2020-09-15  2020-09-16   
1   2020-09-12  2020-09-13  2020-09-14  2020-09-15  2020-09-16  2020-09-17   
2   2020-09-13  2020-09-14  2020-09-15  2020-09-16  2020-09-17  2020-09-18   
3   2020-09-14  2020-09-15  2020-09-16  2020-09-17  2020-09-18  2020-09-19   
4   2020-09-15  2020-09-16  2020-09-17  2020-09-18  2020-09-19  2020-09-20   
5   2020-09-16  2020-09-17  2020-09-18  2020-09-19  2020-09-20  2020-09-21   
6   2020-09-17  2020-09-18  2020-09-19  2020-09-20  2020-09-21  2020-09-22   
7   2020-09-18  2020-09-19  2020-09-20  2020-09-21  2020-09-22  2020-09-23   
8   2020-09-19  2020-09-20  2020-09-21  2020-09-22  2020-09-23  2020-09-24   
9   2020-09-20  2020-09-21  2020-09-22  2020-09-23  2020-09-24  2020-09-25   
10  2020-09-21  2020-09-22  2020-09-23  2020-09-24  2020-09-25  2020-09-26   
11  2020-09-22  2020-09-23  2020-09-24  2020-09-25  2020-09

In [8]:
#print(csv_file)
#df_top_value[0]
df_bottom_value_now[0][0]#9:00のmax_wind

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,...,20,17,14,14,12,9,8,18,30,28
1,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,...,12,12,8,9,10,11,15,18,26,29
2,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,...,15,26,24,26,28,26,22,20,22,21
3,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,...,31,29,26,23,24,18,17,9,11,12
4,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,...,24,24,24,19,18,19,13,8,10,13
5,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,...,17,16,9,12,15,16,11,18,22,21
6,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,...,13,8,8,22,24,21,15,15,14,16
7,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,...,22,20,22,20,22,20,16,17,18,17
8,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,...,20,20,20,20,21,21,16,13,13,10
9,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,...,21,20,17,18,19,16,14,13,13,8


In [9]:
#time_check関数で帰ってくるkeyでtime_dictを使う
time_dict={
           "7:00":[df_bottom_value,df_top_value],
           "9:00":[df_bottom_value_now[0],df_top_value_now[0]],
           "11:00":[df_bottom_value_now[1],df_top_value_now[1]],
           "13:00":[df_bottom_value_now[2],df_top_value_now[2]],
           "15:00":[df_bottom_value_now[3],df_top_value_now[3]],
           "17:00":[df_bottom_value_now[4],df_top_value_now[4]]
                    }
x=time_dict["9:00"]
x_b=x[0]
print(x_b)
#安栄観光の教師データによって使うdfを変更しなければいけないため
#0 -> bottom
#1 -> top
route_selct={
    "taketomi_route" : 0,
    "kurosima_route" : 0,
    "kohama_route" : 1,
    "iriomote_uehara_route" : 1,
    "hatoma_route" : 1,
    "iriomote_ohara_route" : 0,
   "hateruma_route" : 0
}

[            0           1           2           3           4           5   \
0   2020-09-11  2020-09-12  2020-09-13  2020-09-14  2020-09-15  2020-09-16   
1   2020-09-12  2020-09-13  2020-09-14  2020-09-15  2020-09-16  2020-09-17   
2   2020-09-13  2020-09-14  2020-09-15  2020-09-16  2020-09-17  2020-09-18   
3   2020-09-14  2020-09-15  2020-09-16  2020-09-17  2020-09-18  2020-09-19   
4   2020-09-15  2020-09-16  2020-09-17  2020-09-18  2020-09-19  2020-09-20   
5   2020-09-16  2020-09-17  2020-09-18  2020-09-19  2020-09-20  2020-09-21   
6   2020-09-17  2020-09-18  2020-09-19  2020-09-20  2020-09-21  2020-09-22   
7   2020-09-18  2020-09-19  2020-09-20  2020-09-21  2020-09-22  2020-09-23   
8   2020-09-19  2020-09-20  2020-09-21  2020-09-22  2020-09-23  2020-09-24   
9   2020-09-20  2020-09-21  2020-09-22  2020-09-23  2020-09-24  2020-09-25   
10  2020-09-21  2020-09-22  2020-09-23  2020-09-24  2020-09-25  2020-09-26   
11  2020-09-22  2020-09-23  2020-09-24  2020-09-25  2020-09-26 

In [11]:
from datetime import datetime as dt
#img_time_com = ["08:00:00","10:00:00","12:00:00","14:00:00","16:00:00"]
val_time_com = [800,1000,1200,1400,1600]
now_time = ["9:00","11:00","13:00","15:00","17:00"]
dir_label = dir + 'route/taketomi_route/isigaki_dep/' + csv_file[12]

def time_check(row,val_time_com,img_time):
    """
    安栄観光のデータの出発時間により参照するファイルを変更する
    時間によって処理を変更する
    
    paramenters
    -----------
    df : DataFrame
        安栄観光のcsvをdfにしたもので中身
        列名["Time","Label","Date"]
        要素[07:50,0,2020-08-02]
    val_time_com : list[str]
        Timeの比較に必要な時間の配列([800,1000,1200,1400,1600])
    now_time : list[str]
        比較した結果now_valueのpath記述に必要な時間の文字列("9:00","11:00","13:00","15:00","17:00")
    
    return
    ------
    key : str
        安観光データの時間によって参照するwindy_valueのdfを変更するためのkeyを返す
    """
    #df[0] = pd.to_datetime(df[0]) 


    for i in range(len(val_time_com)):    #出発時間の比較のために5回回す

        int_time = re.sub(':', '',row[1])
        #print(int_time)

        if int(int_time) < val_time_com[i]:    #航路出発時間 < 数値の取得時間(7:00~17:00)
            if 0 == i:
                #print(int_time)
                key="7:00"
                print(key)
            else:
                key = now_time[i-1]
                #print(img_time_com[i],"")
                print(key,"key")

            break
        elif 4 == i:    #4はimg_time_comのすべてにかからなかった場合最終ループであるのことの確認
            key = now_time[i]
            print(key,"key")
        #print("1行")

    return key
df = pd.read_csv(dir_label, header=None)
#テスト
for row in df.itertuples():    #dfを一行ずつ回す
    print(row)
    time_check(row,val_time_com,now_time)
    

Pandas(Index=0, _1='07:30', _2=0, _3='2020-08-14')
7:00
Pandas(Index=1, _1='08:30', _2=0, _3='2020-08-14')
9:00 key
Pandas(Index=2, _1='09:30', _2=0, _3='2020-08-14')
9:00 key
Pandas(Index=3, _1='10:30', _2=0, _3='2020-08-14')
11:00 key
Pandas(Index=4, _1='11:30', _2=0, _3='2020-08-14')
11:00 key
Pandas(Index=5, _1='13:00', _2=0, _3='2020-08-14')
13:00 key
Pandas(Index=6, _1='14:00', _2=0, _3='2020-08-14')
15:00 key
Pandas(Index=7, _1='15:00', _2=0, _3='2020-08-14')
15:00 key
Pandas(Index=8, _1='16:00', _2=0, _3='2020-08-14')
17:00 key
Pandas(Index=9, _1='17:00', _2=0, _3='2020-08-14')
17:00 key
Pandas(Index=10, _1='18:00', _2=0, _3='2020-08-14')
17:00 key


In [18]:
def value_merge(df_value,date_str,df_y,value_colum):
    """
    date+valueのdfをループさせ任意の日付のvalueをdf_merge_1に結合させていく関数
    
    prmeters
    --------
    df_value : dataframe
        date+valueのdf
    date_str : str
        日付のstr
    df_y : dataframe
        安栄観光のデータ1行分
    value_colum : int
        時間によって処理を変更しなけらばならない
    """
    for j, l in enumerate(df_value):                              #valueのdfの配列がdf_value

        l_col_dup = list(l.columns)                               #カラムの列をリストで取得
        new_columns = [x for x in range(0, len(l_col_dup), 1)]    #0~カラム数分の配列作成
        l.columns = new_columns                                   #カラムの入れ替え
        #print(df_top[df_top[0] == date_str])
        try:                                                      #windy_valueにその日付がない場合の処理
            #df_xは特徴量
            df_x = l[l[0] == date_str]              #任意の日付の行を0列から抽出(windy_valueの値)
        except:
            continue                                #日付が無い場合はとばす

        df_x = df_x.reset_index(drop=True)
        df_y = df_y.reset_index(drop=True)
        #print(df_x)
        print(j)
        if j == 0:
            #df_x[value_colum]は時間によって変更しなければいけない
            df_result = pd.concat([df_y, df_x[value_colum]], axis=1)
            dfre_col_dup = list(df_result.columns) 
            new_columns = [x for x in range(0, len(dfre_col_dup), 1)]    #0~カラム数分の配列作成
            #print(new_columns)
            df_result.columns = new_columns
            df_merge = df_result

        elif j != 0:
            #df_x[value_colum]は時間によって変更しなければいけない
            #print(df_x)
            #print(df_x[value_colum])
            df_merge = pd.concat([df_merge, df_x[value_colum]], axis=1)
            dfre_col_dup = list(df_merge.columns) 
            new_columns = [x for x in range(0, len(dfre_col_dup), 1)]    #0~カラム数分の配列作成
            #print(new_columns)
            df_merge.columns = new_columns
            df_merge_1 = df_merge
            #print(df_merge_1)

    return df_merge_1

In [19]:
def new_col(df_merge):
    dfre_col_dup = list(df_merge.columns) 
    new_columns = [x for x in range(0, len(dfre_col_dup), 1)]    #0~カラム数分の配列作成
    df_merge.columns = new_columns
    return df_merge

In [38]:
#日付の計算を行うための
import pandas.tseries.offsets as offsets

"""
3番
dir_labelは安栄観光のデータ
たぶんここでwindyのほうに日付がない場合の処理を入れないといけない
時間があっていると言う仮定で実行する関数
"""

def label_df(dir_label,value_colum,df_value_dic) :
    """
    日付ごとの教師データをdfに変換
    
    parameters
    ---------------
    dir_label : str
        教師データのあるcsvファイルのパス
    value_colum : int
        時間によって取得するwindy_valueのカラムが違うため(7:00->12)
    df_value : dataframe
        windy_valueの
    """
    df = pd.read_csv(dir_label, header=None)    #安栄観光の教師データ
    df[2]= pd.to_datetime(df[2])                #日付データをdatetime型に(2020-08-14)
    #print(df)
    for i, row in enumerate(df.itertuples()):   #安栄観光のデータ数の分ループ(07:30  0 2020-08-14)
        #print(row)
        print(row[3]-offsets.Day(days_ago))
        #print(type(row[3]))
        date_str = str(row[3]-offsets.Day(days_ago))
        date_str = re.sub('(\d*-\d*-\d*) (\d*:\d*:\d*)', '\\1', date_str)
        #print(date_str)
        #df_yは教師ラベル
        df_y = df.query('index == @i')                            #i番目のindex(安栄観光のdfの1行)
        
        val_time_com = [800,1000,1200,1400,1600]
        now_time = ["9:00","11:00","13:00","15:00","17:00"]
        key = time_check(row,val_time_com,now_time)                   #安栄観光データの時間列をチェックして時間を返す
        df_value_x = df_value_dic[key]                                #辞書からkey(時間[7:00])を参照してwindy_valueのdfを受け取る
        #---
        #7:00の時に角度などの特徴量も参照しないといけないため
        if key == "7:00":
            df_value_x_7 = df_value_dic["9:00"]
            route_dic_7 = route_selct["taketomi_route"]                     #航路がtopかbottomかを決定する(0または1)
            df_value_7 = df_value_x_7[route_dic_7]
            df_merge_7 = value_merge(df_value_7,date_str,df_y,value_colum)  #9:00の時のvalue
            print(df_merge_7)
        #---
        print(len(df_value_x))
        #今はテストで竹富航路を代入　関数の引数でもらってくる予定
        route_dic = route_selct["taketomi_route"]                     #航路がtopかbottomかを決定する(0または1)
        df_value = df_value_x[route_dic]
        #print(df_value)
        #関数にした
        df_merge_1 = value_merge(df_value,date_str,df_y,value_colum)

        if i == 0:
            if key == "7:00":
                #print(df_merge_7[4])
                df_merge_1 = pd.concat([df_merge_1, df_merge_7[[4,6,10]]], axis=1)
                df_merge_1 = new_col(df_merge_1)
                print(df_merge_1)
                df_merge_1.reindex(columns=[0,1,2,3,9,4,10,5,6,7,11,8])
            df_merge_end = df_merge_1
            
        elif i != 0:
            if key == "7:00":
                df_merge_1 = pd.concat([df_merge_1, df_merge_7[4,6,10]], axis=1)
                df_merge_1 = new_col(df_merge_1)
                df_merge_1.reindex(columns=['0','1','2','3','9','4','10','5','6','7','11','8'])
            df_merge_end = pd.concat([df_merge_end, df_merge_1])
        
        #df_merge = df_mer(i,df_result,df_merge)
        #print(row.point)
        #if row[0] < 
    return df_merge_end

#何日前のデータを使用するか
days_ago = 1

#csv_file[12] ←今の時点ではforを回していない
dir_label = dir + 'route/taketomi_route/isigaki_dep/' + csv_file[127]
print(dir_label)
value_colum = 12    #今は例として7:00
#df_value = df_top_value[0]   #windy_valueのdfの中のmax_wind
df_value = time_dict   #windy_valueのdfが格納された辞書

df_result_x = label_df(dir_label,value_colum,df_value)
#print(df_result)
df_result_x

../data/route/taketomi_route/isigaki_dep/2020-12-07.csv
2020-12-06 00:00:00
7:00
0
1
2
3
4
5
6
7
8
      0   1          2   3   4    5   6    7   8    9   10  11
0  07:30   0 2020-12-07  34  40  1.9  96  9.7   6  2.4  56  21
2
0
1
2
3
4
5
      0   1          2   3    4    5   6    7   8   9   10  11
0  07:30   0 2020-12-07  36  2.2  9.9   6  3.6  27  40  96  56
2020-12-06 00:00:00
9:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
9:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
11:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
11:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
13:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
15:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
17:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
17:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
17:00 key
2
0
1
2
3
4
5
6
7
8
2020-12-06 00:00:00
17:00 key
2
0
1
2
3
4
5
6
7
8


,0,1,2,3,4,5,6,7,8,9,10,11
0,07:30,0,2020-12-07,36,2.2,9.9,6,3.6,27,40.0,96,56
0,08:30,0,2020-12-07,34,40.0,1.9,96,9.7,6,2.4,56,21
0,09:30,0,2020-12-07,34,40.0,1.9,96,9.7,6,2.4,56,21
0,10:30,0,2020-12-07,34,40.0,1.9,96,9.7,6,2.4,56,21
0,11:30,0,2020-12-07,34,40.0,1.9,96,9.7,6,2.4,56,21
0,13:00,0,2020-12-07,31,40.0,1.8,102,9.9,6,2.5,32,23
0,14:00,0,2020-12-07,31,40.0,1.8,102,9.9,6,2.5,32,23
0,16:00,0,2020-12-07,33,52.0,1.5,104,8.7,18,2.7,36,23
0,16:30,0,2020-12-07,33,52.0,1.5,104,8.7,18,2.7,36,23
0,17:00,0,2020-12-07,33,52.0,1.5,104,8.7,18,2.7,36,23


In [257]:
print(csv_file[127])

2020-12-07.csv


In [124]:
#ここで安栄観光のcsvファイル一つ分のマージを行う
def df_mer(i,df_result):
    if i == 0:
        df_merge = df_result
    elif i != 0:
        df_merge = pd.concat([df_merge, df_result])
    return df_merge

In [91]:
import pandas as pd

#現在の最大表示列数の出力
pd.get_option("display.max_columns")

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 100)

100

In [122]:
str_1 = "08:00:00"
pd.to_datetime(str_1, format="%H:%M:%S")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
df_result

Timestamp('1900-01-01 08:00:00')

,0,1,2,3
0,07:30,0,2020-08-14,9


Pandas(Index=0, _1='07:30', _2=0, _3='2020-08-14')
windy_value
Pandas(Index=1, _1='08:30', _2=0, _3='2020-08-14')
9:00 key
Pandas(Index=2, _1='09:30', _2=0, _3='2020-08-14')
9:00 key
Pandas(Index=3, _1='10:30', _2=0, _3='2020-08-14')
11:00 key
Pandas(Index=4, _1='11:30', _2=0, _3='2020-08-14')
11:00 key
Pandas(Index=5, _1='13:00', _2=0, _3='2020-08-14')
13:00 key
Pandas(Index=6, _1='14:00', _2=0, _3='2020-08-14')
15:00 key
Pandas(Index=7, _1='15:00', _2=0, _3='2020-08-14')
15:00 key
Pandas(Index=8, _1='16:00', _2=0, _3='2020-08-14')
17:00 key
Pandas(Index=9, _1='17:00', _2=0, _3='2020-08-14')
17:00 key
Pandas(Index=10, _1='18:00', _2=0, _3='2020-08-14')
17:00 key


['2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14',
 '2020-08-14']

In [26]:
#print(df_top[df_top.columns.duplicated(keep=False)])
l_col_dup = list(df_top.columns)
print(len( l_col_dup))

df_result

74


,0,1,2,0,1,2,3,4,5,6,...,64,65,66,67,68,69,70,71,72,73
0,08:30,0,2020-08-14,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,...,11,13,12,11,11.0,14.0,15.0,11.0,10.0,11.0


In [30]:
import pandas as pd
from pandas import Series, DataFrame
# データ1の準備
attri_data1 = {'ID':['100','101','102','105','106']
        ,'city':['Tokyo','Chiba','Kyoto','Gunma','Tokyo']
        ,'birth_year':[1991,1992,1985,1996,1981]
        ,'name':['Yamada','Sato','Suzuki','Kitamura','Aoki']}
data_frame1 = DataFrame(attri_data1)

# データ2の準備
attri_data2 = {'ID':['100','101','102','103','104']
        ,'math':[34,77,45,81,98]
        ,'English':[47,64,16,53,37]
        ,'sex':['F','M','F','F','M']
        ,'index_num':[0,1,2,3,4]}
data_frame2 = DataFrame(attri_data2)

In [31]:
data_frame1
data_frame2

,ID,math,English,sex,index_num
0,100,34,47,F,0
1,101,77,64,M,1
2,102,45,16,F,2
3,103,81,53,F,3
4,104,98,37,M,4


In [32]:
df_result_1 = pd.concat([data_frame1, data_frame2["index_num"]], axis=1)######

In [35]:
print(df_result_1[['ID','city']])

    ID   city
0  100  Tokyo
1  101  Chiba
2  102  Kyoto
3  105  Gunma
4  106  Tokyo
